<a href="https://colab.research.google.com/github/sailkargutkar/R-Projects/blob/HELBNTODR100382/HELBNTODR100382_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install contractions
!pip install beautifulsoup4

import numpy as np 
import pandas as pd
from tqdm import tqdm
import copy
import re
import nltk
import tensorflow_hub as hub
import tensorflow as tf
import contractions
import unicodedata
import pathlib
import os
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from bs4 import BeautifulSoup
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.optimizers import Adamax, Adam
from keras import layers
from sklearn.metrics import f1_score, precision_score
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import GlobalMaxPooling1D,Conv1D,Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from sklearn.feature_extraction.text import TfidfVectorizer 

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

     |████████████████████████████████| 317kB 4.7MB/s 
     |████████████████████████████████| 245kB 21.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81702 sha256=5543866651a7a920a7e8bd5e882ddaabac9bab73348bac3c98c8edc5c671614d
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
# The Universal Sentence Encoder encodes text into high dimensional vectors that can be used for text classification, 
# semantic similarity, clustering, and other natural language tasks. 
# The pre-trained Universal Sentence Encoder is publicly available in Tensorflow-hub. 
# It comes with two variations i.e. one trained with Transformer encoder and other trained with Deep Averaging Network (DAN).
universal_sentence_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [5]:
data = pd.read_csv("/content/agr_en_train.csv")
data.columns = ['Corpus','Text','Label']

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11998 entries, 0 to 11997
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Corpus  11998 non-null  object
 1   Text    11998 non-null  object
 2   Label   11998 non-null  object
dtypes: object(3)
memory usage: 281.3+ KB


In [7]:
dataText = data['Text']
dataLabel = data['Label']

In [8]:
# Remove all numbers from Text column
data['Text'] = data['Text'].str.replace('\d+', '')

# Remove any type of alphanumeric character from our dataset.
data.Text.str.replace('[^a-zA-Z0-9]', '')

# Remove any commas, dollar symbol, double quotes, single quotes, question mark and single letter from our dataset.

# Remove multiple space fromour dataset.
data['Text'] = data['Text'].replace('\s+', ' ', regex=True)
data['Text'] = data['Text'].str.replace(',', '')
data['Text'] = data['Text'].str.replace('$', '')
data['Text'] = data['Text'].str.replace('"', '')
data['Text'] = data['Text'].str.replace('?', '')
data['Text'] = data['Text'].str.replace("'", "")
data['Text'] = data['Text'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
data['Text'] = data['Text'].str.replace("/", "")
data['Text'] = data['Text'].str.replace("*", "")
data['Text'] = data['Text'].str.replace("+", "")
data['Text'] = data['Text'].str.replace(":", "")
data['Text'] = data['Text'].str.replace("(", "")
data['Text'] = data['Text'].str.replace(")", "")
data['Text'] = data['Text'].str.replace("[", "")
data['Text'] = data['Text'].str.replace("]", "")

# Remove punctuation from our detaset.
data['Text'] = data['Text'].str.replace('[^\w\s]','')

In [10]:
train_text = data['Text'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

train_label = np.asarray(pd.get_dummies(data.Label), dtype = np.int8)

In [11]:
train_label

array([[0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0]], dtype=int8)

In [65]:
print(len(train_text))
print(len(train_label))

11998
11998


In [66]:
train_texts, test_texts, y_train, y_test =\
  train_test_split(
    train_text,
    train_label,
    test_size=.1,
    random_state=10
  )

In [73]:
print(train_texts.shape)
print(test_texts.shape)
print(y_train.shape)
print(y_test.shape)
# y_train

(10798, 1)
(1200, 1)
(10798, 3)
(1200, 3)


In [74]:
train_texts

array([['Wise men dont speak at every conversation they speak the right words at the right time and he spoke wht is true'],
       ['dnt teach ground realitythe govt hv to tell how much time they need to change the lawwe need action not assurance'],
       ['Hi Sonia would like to know whether to hold Ashok Leyland or Sell it'],
       ...,
       ['First stop the black money in politics dont give election tickets to criminals Clean your own house first before trying to clean others house Gov has the list of names of people having money in Swiss banks and Panama files they are too big corporates who gives lots of their black money to all political parties including BJP No one can touch them Demonitization cleans the black money from middle class business and not from big corporates who hold assents in and '],
       ['Tesla market cap more than FORD GM is it threat to Indian car maker also'],
       ['Where is the outrage for deaths of our Jawans Where are offers of bangles for PM by B

In [75]:
test_texts

array([['No bdy knows about villagers they going to dead  why he is great pm give me reason'],
       ['Demonitisation is historicand is best oppurtunity to take thecountry tofuture or to the botom its great oppurtunity historical'],
       ['Congrats Rahul mummy will be soo happy with ur joke best comedian in the Parliment'],
       ...,
       ['If half of the amount was invested earlier for all the athletes then it surely fetches more no And golds too Ny ways good move'],
       ['Move is gud but pl ensure atm has sufficient cash'],
       ['When guys study Do really study ']], dtype=object)

In [78]:
X_train = []
for r in tqdm(train_texts):
  emb = universal_sentence_encoder(r)
  text_emb = tf.reshape(emb, [-1]).numpy()
  X_train.append(text_emb)
X_train = np.array(X_train)

X_test = []
for r in tqdm(test_texts):
  emb = universal_sentence_encoder(r)
  text_emb = tf.reshape(emb, [-1]).numpy()
  X_test.append(text_emb)
X_test = np.array(X_test)
print(X_train.shape, y_train.shape)\

100%|██████████| 1200/1200 [00:02<00:00, 417.48it/s]

(10798, 512) (10798, 3)


In [80]:
print(X_train.shape, y_train.shape)\

(10798, 512) (10798, 3)


In [81]:
# Split training data and testing data
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=10)

In [124]:
# A convolutional neural network (CNN) is a specific type of artificial neural network that uses perceptrons, 
# a machine learning unit algorithm, for supervised learning, 
# to analyze data. CNNs apply to image processing, natural language processing and other kinds of cognitive tasks.
model = Sequential()
model.add(layers.Embedding(input_dim=10798, 
                           output_dim=200, 
                           input_length=512))
model.add(layers.Conv1D(filters=64, kernel_size=8, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(3, activation='relu'))

In [125]:
# summary() function is a generic function used to produce result summaries of the results of various model fitting functions. 
# The function invokes particular methods which depend on the class of the first argument.
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 512, 200)          2159600   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 505, 64)           102464    
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 252, 64)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 16128)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                161290    
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 33        
Total params: 2,423,387
Trainable params: 2,423,387
Non-trainable params: 0
____________________________________________

In [126]:
# function returns the specified source as a code object, ready to be executed
# compiling out model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [127]:
X_train.shape

(10798, 512)

In [128]:
y_train.shape

(10798, 3)

In [129]:
history = model.fit(
    X_train, y_train,
    epochs=3,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=True
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 9718 samples, validate on 1080 samples
Epoch 1/3
9718/9718 [==============================] - 104s 11ms/step - loss: 6.1253 - accuracy: 0.4164 - val_loss: 5.8762 - val_accuracy: 0.4537
Epoch 2/3
9718/9718 [==============================] - 102s 11ms/step - loss: 6.1245 - accuracy: 0.4189 - val_loss: 5.8745 - val_accuracy: 0.4537
Epoch 3/3
9718/9718 [==============================] - 102s 11ms/step - loss: 6.1245 - accuracy: 0.4189 - val_loss: 5.8751 - val_accuracy: 0.4537


In [130]:
model.evaluate(X_test, y_test)

1200/1200 [==============================] - 3s 2ms/step


[6.246852906545003, 0.40833333134651184]

In [135]:
y_pred = model.predict(X_train[:10])
print(y_pred)

# y_pred1  = model.predict(X_test)
# y_pred1
# y_pred2 = np.argmax(y_pred1, axis=1)
# y_test2 = np.argmax(y_test, axis=1)

# cm=confusion_matrix(y_test2,y_pred2)
# print(cm)

# y_pred1  = model.predict(X_test)
# y_pred2 = np.argmax(y_pred1, axis=1)
# y_test2 = np.argmax(y_test, axis=1)
# print(y_pred1)
# print(y_test)

# score = f1_score(y_test2,y_pred2,average="micro")
# print("F1 score : ", score)

[[0.       4.062939 2.155662]
 [0.       4.062939 2.155662]
 [0.       4.062939 2.155662]
 [0.       4.062939 2.155662]
 [0.       4.062939 2.155662]
 [0.       4.062939 2.155662]
 [0.       4.062939 2.155662]
 [0.       4.062939 2.155662]
 [0.       4.062939 2.155662]
 [0.       4.062939 2.155662]]
